In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.default.utils import keys_by_value, normalize_attrs
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

from profiling_utils import *

In [2]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
g = Neo4jGraph(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [24]:
# here we clear the db
res = g.clear()

In [25]:
nodes = [
    ("a", {"name": "EGFR", "state": "p"}),
    ("b", {"name": "BND"}),
    ("c", {"name": "Grb2", "aa": "S", "loc": 90}),
    ("d", {"name": "SH2"}),
    ("e", {"name": "EGFR"}),
    ("f", {"name": "BND"}),
    ("g", {"name": "Grb2"}),
    ("h", {"name": "WAF1"}),
    ("i", {"name": "BND"}),
    ("j", {"name": "G1-S/CDK", "state": "p"}),
    "k", "l", "m"
    ]
edges = [
    ("a", "b", {"s": "p"}),
    ("d", "b", {"s": "u"}),
    ("d", "c"),
    ("e", "f", {"s": "p"}),
    ("g", "f", {"s": "u"}),
    ("h", "i"),
    ("i", "h"),
    ("j", "h", {"act": {1}}),
    ("j", "i", {"act": {2}}),
    ("k", "l"),
    ("l", "k"),
    ("l", "m"),
    ("m", "l"),
    ("k", "m"),
    ("m", "k"),
    ("e", "b", {"s": "u"})
    ]
g.add_nodes_from(nodes)
g.add_edges_from(edges)

#### Cypher query for cloning (IgnoreNaming=True)

In [ ]:
print("Here we ignore pretty naming of new clones\n")
clone2_res = g.clone_node('a', ignore_naming=True)
print("Created clone by the name: ", clone2_res)
print("Properties of the resulting node: ")
print(g.get_node(clone2_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone2_res, 'b'))

#### Cypher query for cloning

In [ ]:
clone1_res = g.clone_node('a')
print("Created clone by the name: ", clone1_res)
print("Properties of the resulting node: ")
print(g.get_node(clone1_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone1_res, 'b'))

### Profiling clone query

In [20]:
node = "a"
query = "PROFILE\n" +\
        match_node('x', node) +\
        cloning_query(
            original_var='x',
            clone_var='new_node',
            clone_id=node,
            clone_id_var='uid',
            ignore_naming=True)[0] +\
        return_vars(['uid'])
print(query)
res = g.execute(query).summary()

PROFILE
MATCH (x:node { id : 'a' })
WITH [] as sucIgnore, [] as predIgnore, x 
// match successors and out-edges of a node to be cloned
OPTIONAL MATCH (x)-[out_edge:edge]->(suc) 
WHERE NOT suc.id IS NULL AND NOT suc.id IN sucIgnore
WITH collect({neighbor: suc, edge: out_edge}) as suc_maps, predIgnore, x 
// match predecessors and in-edges of a node to be cloned
OPTIONAL MATCH (pred)-[in_edge:edge]->(x) 
WHERE NOT pred.id IS NULL AND NOT pred.id IN predIgnore
WITH collect({neighbor: pred, edge: in_edge}) as pred_maps, x, suc_maps 
// create a node corresponding to the clone
CREATE (new_node:node) 
WITH new_node, toString(id(new_node)) as uid, x.id as original_old, x, pred_maps, suc_maps 
// set the id property of the original node to NULL
SET x.id = NULL
// copy all the properties of the original node to the clone
SET new_node = x
// set id property of the clone to neo4j-generated id
SET new_node.id = toString(id(new_node)), new_node.count = NULL
// set back the id property of the origi

In [21]:
print(type(res))

<class 'neo4j.v1.result.BoltStatementResultSummary'>


In [22]:
print(res.metadata["profile"])

{'args': {'PageCacheHitRatio': 0.0, 'planner-impl': 'IDP', 'runtime': 'INTERPRETED', 'runtime-impl': 'INTERPRETED', 'PageCacheMisses': 0, 'version': 'CYPHER 3.3', 'EstimatedRows': 1.0677899723724409, 'planner': 'COST', 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0}, 'pageCacheMisses': 0, 'children': [{'args': {'PageCacheHitRatio': 0.0, 'Expressions': '{uid :   uid@1066}', 'PageCacheMisses': 0, 'EstimatedRows': 1.0677899723724409, 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0}, 'pageCacheMisses': 0, 'children': [{'args': {'PageCacheHitRatio': 0.0, 'PageCacheMisses': 0, 'EstimatedRows': 1.0677899723724409, 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0}, 'pageCacheMisses': 0, 'children': [{'args': {'PageCacheHitRatio': 0.0, 'PageCacheMisses': 0, 'EstimatedRows': 1.0677899723724409, 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0}, 'pageCacheMisses': 0, 'children': [{'args': {'PageCacheHitRatio': 0.0, 'PageCacheMisses': 0, 'EstimatedRows': 1.0677899723724409, 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0},

In [12]:
for p in res.profile:
    print(p)

ProduceResults
['x', '  uid@637', 'original_old', '  uid@1066', 'suc_maps', 'pred_maps', 'new_node', 'uid']
{'PageCacheHitRatio': 0.0, 'planner-impl': 'IDP', 'runtime': 'INTERPRETED', 'runtime-impl': 'INTERPRETED', 'PageCacheMisses': 0, 'version': 'CYPHER 3.3', 'EstimatedRows': 1.0677899723724409, 'planner': 'COST', 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0}
[ProfiledPlan(operator_type='Projection', identifiers=['x', '  uid@637', 'original_old', '  uid@1066', 'suc_maps', 'pred_maps', 'new_node', 'uid'], arguments={'PageCacheHitRatio': 0.0, 'Expressions': '{uid :   uid@1066}', 'PageCacheMisses': 0, 'EstimatedRows': 1.0677899723724409, 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0}, children=[ProfiledPlan(operator_type='Foreach', identifiers=['x', '  uid@637', 'original_old', '  uid@1066', 'suc_maps', 'pred_maps', 'new_node'], arguments={'PageCacheHitRatio': 0.0, 'PageCacheMisses': 0, 'EstimatedRows': 1.0677899723724409, 'DbHits': 0, 'Rows': 1, 'PageCacheHits': 0}, children=[ProfiledPlan(o

In [23]:
number_of_DbHits(res.profile)

0
0
0
0
0
0
0
1
1
1
10
2
0
1
0
2
0
0
0
0
0
0
0
13
14


45

#### Time to match a node with its id

In [ ]:
node = "a"
query = match_node(node, node) + return_vars(node)
print(g.execution_time(query))

#### Time to add a node

In [ ]:
node = "x"
attrs = {"act": {1}}
normalize_attrs(attrs)
query = create_node(node, node, 'new_id', attrs)[0]
print(g.execution_time(query))

#### Time to remove a node

In [ ]:
node = "x"
query = match_node(node, node) + delete_nodes_var(node)
print(g.execution_time(query))

#### Time to add an edge

In [ ]:
source = "a"
target = "c"
attrs = {"act": {1}}
normalize_attrs(attrs)
query = match_nodes({
            source: source,
            target: target
        })
query += create_edge(source, target, attrs)
print(g.execution_time(query))

#### Time to delete an edge

In [ ]:
source = "a"
target = "c"
query = match_edge(source, target, source, target, 'r') 
query += delete_edge_var('r')
print(g.execution_time(query))

#### Time to clone a node

In [ ]:
node = "a"
clone = "a_clone"
query =\
            match_node('x', node) +\
            cloning_query(
                original_var='x',
                clone_var='new_node',
                clone_id=clone,
                clone_id_var='uid')[0] +\
            return_vars(['uid'])
print(g.execution_time(query))